# 面向过程的有限元编程

我们首先将前几节的代码汇总一下，并将其分为三个部分：有限元前处理、有限元求解器和有限元后处理。

有限元前处理（由用户输入）：

In [23]:
import numpy as np
import pygmsh

# 定义网格
with pygmsh.geo.Geometry() as geom:
    geom.add_polygon([[0.0, 0.0],[1.0, 0.0],[1.0, 1.0],[0.0, 1.0],],mesh_size=0.5,)
    mesh = geom.generate_mesh()
nodes=mesh.points
elements=mesh.cells_dict['triangle']

# 定义材料
E=10.0
nv=0.3

# 定义载荷
x_fix={0:0.0,2:0.0,3:0.0}
y_fix={0:0.0,2:0.0,3:0.0}
f_given={5:(1.0,0.0)}

有限元求解器：

In [24]:
# 第一步
# 材料本构：刚度矩阵（和切向刚度矩阵）
D=E/(1.0-nv**2)*np.array([
    [1.0,nv,0.0], 
    [nv,1.0,0.0], 
    [0.0,0.0,0.5*(1.0-nv)]])

# 第二步
# 生成总体系统：节点位移、节点力和刚度矩阵
# 总体系统大小
len_global=2*len(nodes)
deform=np.zeros(len_global)
force=np.zeros(len_global)
K=np.zeros((len_global,len_global))
#初始化节点位移
for node in x_fix:
    deform[2*node]=x_fix[node]
for node in y_fix:
    deform[2*node+1]=y_fix[node]
#初始化节点力
for node in f_given:
    force[2*node]=f_given[node][0]
    force[2*node+1]=f_given[node][1]
#初始化刚度矩阵
for element in elements:
    x0=nodes[element[0]][0]
    y0=nodes[element[0]][1]
    x1=nodes[element[1]][0]
    y1=nodes[element[1]][1]
    x2=nodes[element[2]][0]
    y2=nodes[element[2]][1]
    area_2=np.linalg.det(np.array([
        [1.0,x0,y0], 
        [1.0,x1,y1], 
        [1.0,x2,y2]]))
    B=1/area_2*np.array([
        [y1-y2,0.0,y2-y0,0.0,y0-y1,0.0], 
        [0.0,x2-x1,0.0,x0-x2,0.0,x1-x0], 
        [x2-x1,y1-y2,x0-x2,y2-y0,x1-x0,y0-y1]])
    K_element=area_2*B.T@D@B
    deform_global_index=np.array([[2*element[i],2*element[i]+1] for i in range(3)],dtype=np.uint64).reshape(-1)
    for i_local,i_global in enumerate(deform_global_index):
        for j_local,j_global in enumerate(deform_global_index):
            K[i_global,j_global]+=K_element[i_local,j_local]

# 第三步
# 生成约化系统：节点位移、节点力和刚度矩阵
# 生成约化系统指标列表
deform_free_index=[]
for node,_ in enumerate(nodes):
    if node not in x_fix:
        deform_free_index.append(2*node)
    if node not in y_fix:
        deform_free_index.append(2*node+1)
# 约化系统大小
len_reduce=len(deform_free_index)
deform_reduce=np.empty(len_reduce)
force_reduce=np.zeros(len_reduce)
K_reduce=np.zeros((len_reduce,len_reduce))
# 约化节点力
for i_reduce,i_global in enumerate(deform_free_index):
    force_reduce[i_reduce]=force[i_global]
# 约化刚度矩阵
for i_reduce,i_global in enumerate(deform_free_index):
    for j_reduce,j_global in enumerate(deform_free_index):
        K_reduce[i_reduce,j_reduce]=K[i_global,j_global]

# 第四步
# 求解约化系统
deform_reduce=np.linalg.solve(K_reduce,force_reduce)

# 第五步
# 更新总体系统：节点位移和节点力
# 更新节点位移
for i_reduce,i_global in enumerate(deform_free_index):
    deform[i_global]=deform_reduce[i_reduce]
# 更新节点力
force=K@deform

有限元后处理（在面向对象版本中详述）：

In [25]:
# 展示计算结果
# 画图、获取数据
# 举一例，展示节点位移
print(deform.reshape(len(nodes),2))

[[ 0.          0.        ]
 [ 0.03611171 -0.00798372]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.03297995  0.00472674]
 [ 0.08560047  0.00172474]
 [ 0.01980262 -0.01199891]
 [ 0.03448744 -0.00319907]
 [ 0.0280644  -0.00439485]
 [ 0.03614977 -0.00554314]
 [ 0.04470738 -0.01290549]
 [ 0.05377041  0.00153482]]


我们对内核部分进行重构，将主干部分完全函数化。按照注释将各部分整理成函数。

第一步

In [26]:
def cal_D(E,nv):
    return E/(1.0-nv**2)*np.array([
        [1.0,nv,0.0], 
        [nv,1.0,0.0], 
        [0.0,0.0,0.5*(1.0-nv)]])
D=cal_D(E=10.0,nv=0.3)

第二步，首先定义以下函数

初始化需要用到的全局变量

In [27]:
def init_global(nodes):
    len_global=2*len(nodes)
    deform=np.zeros(len_global)
    force=np.zeros(len_global)
    K=np.zeros((len_global,len_global))
    return deform,force,K

确定单元中各位移

In [28]:
def get_position_local(element):
    x0=nodes[element[0]][0]
    y0=nodes[element[0]][1]
    x1=nodes[element[1]][0]
    y1=nodes[element[1]][1]
    x2=nodes[element[2]][0]
    y2=nodes[element[2]][1]
    return x0,y0,x1,y1,x2,y2

计算二倍面积

In [29]:
def det_B(x0,y0,x1,y1,x2,y2):
    matrix=np.array([
        [1.0,x0,y0], 
        [1.0,x1,y1], 
        [1.0,x2,y2]])
    det=np.linalg.det(matrix)
    return abs(det)

计算 $B$ 矩阵

In [30]:
def b_matrix(x0,y0,x1,y1,x2,y2,area_2):
    return 1/area_2*np.array([\
        [y1-y2,0.0,y2-y0,0.0,y0-y1,0.0], 
        [0.0,x2-x1,0.0,x0-x2,0.0,x1-x0], 
        [x2-x1,y1-y2,x0-x2,y2-y0,x1-x0,y0-y1]])

单元积分，计算单元刚度矩阵 $K^e$

In [31]:
def element_integrate(B,D,area_2):
    return area_2*B.T@D@B

将单元

In [32]:
def Ke2K(element,K_element,K):
    deform_global_index=np.array([[2*element[i],2*element[i]+1] for i in range(3)],dtype=np.uint64).reshape(-1)
    for i_local,i_global in enumerate(deform_global_index):
        for j_local,j_global in enumerate(deform_global_index):
            K[i_global,j_global]+=K_element[i_local,j_local]

初始化节点位移

In [33]:
def init_deform(x_fix,y_fix):
    for node in x_fix:
        deform[2*node]=x_fix[node]
    for node in y_fix:
        deform[2*node+1]=y_fix[node]
    return deform

初始化节点力

In [34]:
def init_force(f_given):
    for node in f_given:
        force[2*node]=f_given[node][0]
        force[2*node+1]=f_given[node][1]
    return force

调用以上各个函数

In [35]:
deform,force,K=init_global(nodes)
deform=init_deform(x_fix,y_fix)
force=init_force(f_given)
for element in elements:
    # calculate stiffness of element
    x0,y0,x1,y1,x2,y2=get_position_local(element)
    area_2=det_B(x0,y0,x1,y1,x2,y2)
    B=b_matrix(x0,y0,x1,y1,x2,y2,area_2)
    K_element=element_integrate(B,D,area_2)
    # add local Ke into global K
    Ke2K(element,K_element,K)

## 第三步

生成约化系统指标列表

In [36]:
def cal_reduce_map(nodes):
    deform_free_index=[]
    for node,_ in enumerate(nodes):
        if node not in x_fix:
            deform_free_index.append(2*node)
        if node not in y_fix:
            deform_free_index.append(2*node+1)
    return deform_free_index

约化系统大小

In [37]:
def init_reduce(deform_free_index):
    len_reduce=len(deform_free_index)
    deform_reduce=np.empty(len_reduce)
    force_reduce=np.zeros(len_reduce)
    K_reduce=np.zeros((len_reduce,len_reduce))
    return deform_reduce,force_reduce,K_reduce

约化节点力

In [38]:
def reduce_force(force,deform_free_index):
    for i_reduce,i_global in enumerate(deform_free_index):
        force_reduce[i_reduce]=force[i_global]
    return force_reduce

约化刚度矩阵

In [39]:
def reduce_K(K,deform_free_index):
    for i_reduce,i_global in enumerate(deform_free_index):
        for j_reduce,j_global in enumerate(deform_free_index):
            K_reduce[i_reduce,j_reduce]=K[i_global,j_global]
    return K_reduce

调用以上函数

In [40]:
deform_free_index=cal_reduce_map(nodes)
deform_reduce,force_reduce,K_reduce=init_reduce(deform_free_index)
force_reduce=reduce_force(force,deform_free_index)
K_reduce=reduce_K(K,deform_free_index)

In [41]:
force_reduce

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

## 第四步

求解约化系统

In [42]:
deform_reduce=np.linalg.solve(K_reduce,force_reduce)

## 第五步

更新总体系统：节点位移和节点力

In [43]:
def update_variables(deform_reduce,deform_free_index):
    # 更新节点位移
    for i_reduce,i_global in enumerate(deform_free_index):
        deform[i_global]=deform_reduce[i_reduce]
    # 更新节点力
    force=K@deform
    return deform,force
deform,force=update_variables(deform_reduce,deform_free_index)

[ 0.          0.          0.03611171 -0.00798372  0.          0.
  0.          0.          0.03297995  0.00472674  0.08560047  0.00172474
  0.01980262 -0.01199891  0.03448744 -0.00319907  0.0280644  -0.00439485
  0.03614977 -0.00554314  0.04470738 -0.01290549  0.05377041  0.00153482]


In [44]:
deform

array([ 0.        ,  0.        ,  0.03611171, -0.00798372,  0.        ,
        0.        ,  0.        ,  0.        ,  0.03297995,  0.00472674,
        0.08560047,  0.00172474,  0.01980262, -0.01199891,  0.03448744,
       -0.00319907,  0.0280644 , -0.00439485,  0.03614977, -0.00554314,
        0.04470738, -0.01290549,  0.05377041,  0.00153482])